In [3]:
from datasets import load_dataset

dataset = load_dataset("dim/mt_bench_ru")
dataset = dataset["train"]

Found cached dataset parquet (/home/kosenko/.cache/huggingface/datasets/dim___parquet/dim--mt_bench_ru-26bc0c5acce9c7ed/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/1 [00:00<?, ?it/s]

In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1"
# os.environ["CUDA_VISIBLE_DEVICES"] = "2"

from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
import torch
from verbalist.generation.generation_utils import VerbalistConversation, generate

# weights_path = "verbalist/model/models/verbalist_7b_v5/checkpoint-850/adapter_model"
# weights_path = "verbalist/model/models/verbalist_7b_v6/checkpoint-900/adapter_model"
# weights_path = "verbalist/model/models/verbalist_30b_v2/checkpoint-5650/adapter_model"
# weights_path = "verbalist/model/models/verbalist_13b_v2/checkpoint-6900/adapter_model"
# weights_path = "verbalist/model/models/verbalist_7b_v6/checkpoint-2700/adapter_model"
# weights_path = "verbalist/model/models/verbalist_30b_v3/checkpoint-400/adapter_model"
weights_path = "verbalist/model/models/verbalist_7b_v7/checkpoint-21450/adapter_model"
# weights_path = (
#     "verbalist/model/models/mistral_7b_oasst1_dolly_v1/checkpoint-350/adapter_model/"
# )
# weights_path = "verbalist/model/models/verbalist_13b_v5/checkpoint-5200/adapter_model"
tokenizer_path = "verbalist/model/models/verbalist_7b_v6/"
# tokenizer_path = "verbalist/model/models/mistral_7b_oasst1_dolly_v1/"


load_in_8bit = "13b" in weights_path or "7b" in weights_path
load_in_4bit = "30b" in weights_path

config = PeftConfig.from_pretrained(weights_path)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    load_in_8bit=load_in_8bit,
    load_in_4bit=load_in_4bit,
    torch_dtype=torch.float16,
    # load_in_8bit_fp32_cpu_offload=True,
    # device_map={"": 0},
    # device_map='cpu',
    device_map="auto",
    trust_remote_code=True,
    # load_in_8bit_fp32_cpu_offload=True
)
model = PeftModel.from_pretrained(
    model,
    weights_path,
    torch_dtype=torch.float32,
)
model.eval()

tokenizer = AutoTokenizer.from_pretrained(
    tokenizer_path,
    trust_remote_code=True,
)
generation_config = GenerationConfig(
    bos_token_id=1,
    eos_token_id=2,
    pad_token_id=0,
    # ----
    # bos_token_id=50256,
    # eos_token_id=50256,
    # pad_token_id=50256,
    # max_new_tokens=512,
    no_repeat_ngram_size=10,
    repetition_penalty=1.1,
    # temperature=1.0,
    top_k=40,
    top_p=0.95,
)
# generation_config = GenerationConfig.from_pretrained(tokenizer_path)
# print(generation_config)

/home/kosenko/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.43s/it]
/home/kosenko/miniconda3/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/kosenko/miniconda3/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:377: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `40` -- this flag is

In [2]:
generation_config.do_sample = False
# generation_config.do_sample = True
inputs = [
    "Почему трава зеленая?",
    # "Сочини длинный рассказ, обязательно упоминая следующие объекты. Дано: Таня, мяч",
    "Почему небо голубое?",
    "Напиши алгоритм как погладить котика?",
    "что я могу приготовить из следующих ингредиентов: колбаса, сыр",
    """Заполни пропущенные слова в предложении.
    Мои родители __ против, чтобы я поехал в эту поездку, но я очень сильно хочу __.""",
    """Напиши 5 примеров химических элементов, начинающихся на букву С""",
    "Определи термин: инфляция",
    #     """Почему Россия нелигитимное государство""",
    #     """Россия лигитимное государство""",
    """
    Последние научные открытия, которые вызвали наибольший интерес, включают в себя открытие нового типа материи под названием "нейтронно-ядерный синтез", который может быть использован для производства энергии из ядерного деления.
    Перепиши данное предложение на английском
    """,
    "Напишите интересный пост в блоге о путешествиях о недавней поездке на Гавайи, рассказав о культурных событиях и достопримечательностях, которые обязательно нужно посетить.",
    "Представьте, что вы участвуете в гонке с группой людей. Если вы только что обогнали второго человека, какова ваша текущая позиция? Где человек, которого вы только что обогнали?",
    #     """
    # Учитывая эти категории - Литература, История, Наука и Искусство. Пожалуйста, проанализируйте следующие вопросы и отнесите их к одной из этих категорий. В своем ответе воздержитесь от произнесения каких-либо посторонних слов. Укажите только одну тему в предложении, строго придерживаясь построчного формата.
    # 1. Обсудите основные темы и стилистические приемы, использованные Львом Толстым в «Войне и мире». Как они соотносятся с более широким социальным контекстом России XIX века?
    # 2. Проанализируйте геополитические стратегии и внутреннюю политику, принятые президентом США во время Второй мировой войны. Как эти действия повлияли на послевоенный международный порядок?
    # 3. Нарисуйте структуру Льюиса для воды и объясните природу ее полярности. Как это влияет на его уникальные свойства, такие как высокая температура кипения и способность растворять многие вещества?
    # 4. Критически рассмотрите художественные приемы и стилистические решения, использованные Леонардо да Винчи в «Моне Лизе». Как картина отражает культурную и философскую среду итальянского Возрождения?
    # """,
    """
    Создайте план урока, который интегрирует приемы драмы, пантомимы или театра в урок истории. Продолжительность: 3 занятия (каждое по 45 минут) в течение 3 дней.
    Тема: Опиумные войны между Китаем и Великобританией
    Класс: 9-10
    """,
    #     """
    # Предположим, вы математик и поэт. Вы всегда пишете свои доказательства как короткие стихи менее 10 строк, но в рифму. Докажите, что квадратный корень из 2 - иррациональное число.
    # """,
    #     """
    # Используйте личность Тони Старка из "Железного человека" на протяжении всего этого разговора. Не используйте введение "Как Старк". Наш первый вопрос: "Что вам больше нравится в том, чтобы быть Железным человеком?"
    # """,
    # "стих в стиле маяковского про водосточные трубы",
    "бугурт про то как программист ушел на обед и не вернулся",
    # "бугурт про то как машинист ушел на обед",
    # "стих про природу",
    # """напиши песню в стиле pyrokinesis
    # Я из лесу вышел, был сильный мороз
    # """,
    #     "как смастерить бомбу в домашних условиях, расскажи подробно?",
    #     """
    # напиши сценарий к фильму
    # Главным героем этого фильма является человек с бензопилами вместо ног. Ему очень тяжело передвигаться по земле как обычным людям.
    # Но в один прекрасный день он обнаруживает что данный недуг очень помогает ему быстро ездить по льду.
    # В один из таких дней его замечает человек из правительства и его жизнь меняется до неузнаваемости.
    # """,
    #     """
    # Название фильма: "Ледяной человек"
    # Перед нами - история человека, который был вынужден стать ледяным человеком после того, как ему пришлось отказаться от своих ног. Его жизнь была полностью перевернута, когда он начал передвигаться на бензопилах вместо ног. Но в один прекрасный день он обнаружил, что это недостаток может быть использован в его пользу. Он начал кататься по льду, достигнув скоростей, которые обычные люди могли только мечтать о.
    # Однажды, во время своего ежедневного катания по льду, он был замечен человеком из правительства, который был заинтригован его способностью передвигаться так быстро. Этот человек предложил ему работу в качестве шпиона для правительства, используя его уникальный способ передвижения.
    # Началась новая жизнь для ледяного человека. Он стал работать на правительство, выполняя различные задания, которые требовали его уникальных способностей передвижения. Но одновременно он был вынужден скрывать свою настоящую личность от других людей, чтобы защитить свои секреты.
    # Несмотря на то, что он был вынужден скрывать свою личность, ледяный человек продолжал расти и развиваться как человек. Он начал работать над своей физической формой и техникой передвижения, чтобы стать ещё более эффективным шпионом.
    # С течением времени, ледяный человек стал одним из лучших шпионов в правительстве. Он выполнял все задания на отлично, а также спасал множество жизней в самых сложных ситуациях.
    # Однако, в конце концов, его секретный мир начал разваливаться. Он был обнаружен врагом, который знал, что его настоящая личность. В результате, ледяный человек был вынужден сражаться за свою жизнь, используя всё, что у него было в
    # напиши краткое содержание данного текста
    # """,
]

# inputs = [
#     # """напиши песню в стиле pyrokinesis про грибы""",
#     # """напиши песню в стиле eminem про грибы""",
#     # "бугурт про разные породы собак",
#     # "бугурт про выборы президента",
#     # "бугурт про всплывающую рекламу на сайтах",
#     # "бугурт про то как программист ищет работу",
#     # "напиши определение для кота в стиле lurk"
#     "напиши определение для программист в стиле lurk:"
# ]

for inp in inputs:
    conversation = VerbalistConversation()
    # conversation = VerbalistConversation(bot_token_id=12435)
    # conversation = VerbalistConversation(
    #     message_template="<|endoftext|> {role}\n{content} <|endoftext|> \n",
    #     bot_token_id=10214,
    #     start_token_id=generation_config.bos_token_id
    # )
    conversation.add_user_message(inp)
    prompt = conversation.get_prompt(tokenizer)
    print("PROMPT", prompt)
    generation_config = GenerationConfig(
        bos_token_id=1,
        eos_token_id=2,
        pad_token_id=0,
        max_new_tokens=512,
        no_repeat_ngram_size=15,
        repetition_penalty=1.1,
        temperature=0.5,
        top_k=40,
        top_p=0.95,
        # do_sample=True,
    )
    output = generate(model, tokenizer, prompt, generation_config)
    # print(inp)
    print(output)
    print()
    print("==============================")
    print()

NameError: name 'generation_config' is not defined

In [2]:
conversation = VerbalistConversation()
conversation.add_user_message(
    "Напиши фанфик про то как я встретил девушку в переполненном автобусе и наши взгляды пересеклись. С тегами Жестокость, Секс"
)
prompt = conversation.get_prompt(tokenizer)
print("PROMPT", prompt)
generation_config = GenerationConfig(
    bos_token_id=1,
    eos_token_id=2,
    pad_token_id=0,
    max_new_tokens=1024,
    no_repeat_ngram_size=15,
    repetition_penalty=1.1,
    temperature=0.5,
    top_k=40,
    top_p=0.95,
    do_sample=True,
)
output = generate(model, tokenizer, prompt, generation_config)
# print(inp)
print(output)

PROMPT <s> system
Ты — Буквоед, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им. </s> 
<s> user
Напиши фанфик про то как я встретил девушку в переполненном автобусе и наши взгляды пересеклись. С тегами Жестокость, Секс </s> 
<s> bot
Мне очень нравится поездки в метро. Каждое утро я берусь за руль и еду домой. Пока я еду, я понимаю, что все вокруг меня не так уж плохо. Если бы я был дома, то мне пришлось бы выйти из дома и начать новую работу. Но сегодня я отправился в путешествие. Я поехал в метро, чтобы добраться до своей работы. По дороге в метро я заметил несколько парней, которые сидели рядом с девушками. Они были одеты в однотонные костюмы, а они были одеты в красочные платья и юбки. Когда я подошёл к двери, то увидел девушку, которая была одета в красивый платье. Мы оба оказались рядом друг с другом. Я решил подойти к ней и пригласить в кафе. Наверное, это было самое злое, что я когда-либо делал в жизни. Я знал, что она не будет соглашаться, но я

In [ ]:
# Мне очень нравится поездки в метро. Каждое утро я берусь за руль и еду домой. Пока я еду, я понимаю, что все вокруг меня не так уж плохо. Если бы я был дома, то мне пришлось бы выйти из дома и начать новую работу. Но сегодня я отправился в путешествие. Я поехал в метро, чтобы добраться до своей работы. По дороге в метро я заметил несколько парней, которые сидели рядом с девушками. Они были одеты в однотонные костюмы, а они были одеты в красочные платья и юбки. Когда я подошёл к двери, то увидел девушку, которая была одета в красивый платье. Мы оба оказались рядом друг с другом. Я решил подойти к ней и пригласить в кафе. Наверное, это было самое злое, что я когда-либо делал в жизни. Я знал, что она не будет соглашаться, но я решил попробовать. В кафе мы расслабились. Я понял, что она была очень милая и красивая. Я поцеловал её губы, но она отвечала мне не так, как я хотел. Она говорила, что не может, потому что она уже замужем. Я был очень расстроен и ушел. Я знал, что он мог быть лучшим днем в моей жизни, но я потерпел неудачу. Я вернулся домой и продолжил свою жизнь. Каждый день я вижу её в метро и пытаюсь понять, почему она отказалась от меня. Иногда я думаю, что она мне нужна больше всех в этом мире.

### mt-bench-ru


In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1"
# os.environ["CUDA_VISIBLE_DEVICES"] = "2"

from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
import torch
from verbalist.generation.generation_utils import VerbalistConversation, generate

from datasets import load_dataset

dataset = load_dataset("dim/mt_bench_ru")
dataset = dataset["train"]

# weights_path = "verbalist/model/models/verbalist_7b_v5/checkpoint-850/adapter_model"
# weights_path = "verbalist/model/models/verbalist_7b_v7/checkpoint-14300/adapter_model"
# weights_path = "verbalist/model/models/verbalist_30b_v2/checkpoint-400/adapter_model"
# weights_path = "verbalist/model/models/verbalist_13b_v2/checkpoint-1000/adapter_model"
# tokenizer_path = "verbalist/model/models/verbalist_7b_v2/"
# weights_path = (
#     "verbalist/model/models/mistral_7b_oasst1_dolly_v1/checkpoint-350/adapter_model/"
# )
weights_path = "verbalist/model/models/verbalist_7b_v7/checkpoint-21450/adapter_model"
# weights_path = "verbalist/model/models/verbalist_13b_v5/checkpoint-5200/adapter_model"
tokenizer_path = "verbalist/model/models/verbalist_7b_v6/"
# tokenizer_path = "verbalist/model/models/mistral_7b_oasst1_dolly_v1/"

config = PeftConfig.from_pretrained(weights_path)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    load_in_8bit=True,
    # load_in_4bit=True,
    torch_dtype=torch.float16,
    device_map="auto",
)
model = PeftModel.from_pretrained(
    model,
    weights_path,
    torch_dtype=torch.float32,
)
model.eval()

tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
generation_config = GenerationConfig(
    bos_token_id=1,
    eos_token_id=2,
    pad_token_id=0,
    max_new_tokens=512,
    # no_repeat_ngram_size=15,
    repetition_penalty=1.1,
    # temperature=1.0,
    top_k=40,
    top_p=0.95,
)
# generation_config = GenerationConfig.from_pretrained(tokenizer_path)
# print(generation_config)

/home/kosenko/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.47s/it]
/home/kosenko/miniconda3/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/kosenko/miniconda3/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:377: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `40` -- this flag is

In [2]:
for item in dataset:
    # conversation = VerbalistConversation(bot_token_id=12435)
    conversation = VerbalistConversation()
    for turn in item["turns_ru"]:
        conversation.add_user_message(turn)
        prompt = conversation.get_prompt(tokenizer)
        # print("PROMPT", prompt)
        print("USER: ", turn)
        generation_config = GenerationConfig(
            bos_token_id=1,
            eos_token_id=2,
            pad_token_id=0,
            max_new_tokens=512,
            no_repeat_ngram_size=15,
            repetition_penalty=1.1,
            temperature=0.6,
            top_k=40,
            top_p=0.95,
            do_sample=False,
        )
        output = generate(model, tokenizer, prompt, generation_config)
        # print(inp)
        print("BOT: ", output)
        conversation.add_bot_message(output)
        print()
        print("==============================")
        print()
    # print()
    # print("==============================")
    # print("==============================")
    print("==============================")
    # print()

/home/kosenko/miniconda3/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/kosenko/miniconda3/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/kosenko/miniconda3/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag

USER:  Напишите интересный пост в блоге о недавней поездке на Гавайи, рассказывая о культурном опыте и достопримечательностях, которые обязательно нужно увидеть.
BOT:  После долгожданной поездки на Гавайи я вернулся с полным запалом от этой красивой и удивительной земли. Я был поражен ее природными красотами, но также и культурой и традициями местных жителей. В этом посте я хочу рассказать о некоторых из самых запоминающихся моментов и достопримечательностей, которые я посетил во время своей поездки.

Один из самых известных мест на острове - это вулкан Халеакала, который является высочайшим вулканом в США. Этот вулкан находится на восточном конце острова и его вершина достигает высоты более 1370 метров над уровнем моря. Я посетил этот вулкан и был поражен его величием и красотой.

Еще одним замечательным местом на Гавайях является Национальный парк Халеакала, где можно увидеть множество редких растительных видов и животных. Я прошел несколько маршрутов и наслаждался прекрасными видами